## lib

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict

from langchain import PromptTemplate, LLMChain
from langgraph.graph import START, END, MessagesState, StateGraph
from langgraph.checkpoint.memory import InMemorySaver
#from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

## LLM

In [ ]:
import os

silicon_api_key = os.getenv("SILICON_API_KEY")
silicon_base_url = os.getenv("SILICON_BASE_URL")
silicon_llm_model = os.getenv("LLM_MODEL_SLIM")

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# init LLM mod
llm = ChatOpenAI(
    model=silicon_llm_model,
    openai_api_key=silicon_api_key,
    base_url=silicon_base_url,
    temperature=1.2,
    max_tokens=2048,
)

## langgraph

In [ ]:
from typing import List, TypedDict

class AgentState(MessagesState):
    question: str
    generation: str
    documents: List[str]

In [ ]:
def call_model(state: AgentState):
    print("---CALL MODEL---")
    
    print(state["messages"])
    out_message = {"role": "system", "content": "this is a respond"}

    return {"messages": out_message, "question": "Test question", "generation": "Test generation"}


builder = StateGraph(AgentState)
builder.add_node("call_model", call_model)
builder.add_edge(START, "call_model")


In [ ]:
# define memory
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

In [ ]:
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage

In [ ]:
thread_id = "chat-memory-test"
config = {"configurable": {"thread_id": thread_id}}
user_input = "hello"
input_message = HumanMessage(content=user_input)
for chunk in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    print("\n")
    print("---message---")
    for message in chunk["messages"]:
        message.pretty_print()